In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import mode

In [3]:
df = pd.read_csv("./data/online_retail.csv")
df_pre = pd.read_csv('./data/Online_Retail(전처리 후).csv')
df_pre2 = pd.read_csv('./data/df_pre2.csv')
df_retention_rate = pd.read_csv("./data/df_retention_rate.csv")

df_12_description = pd.read_csv("./data/df_12_description.csv")

df_201012 = pd.read_csv("./data/df_201012.csv")
df_not_201012 = pd.read_csv("./data/df_not_201012.csv")

# 월별 신규고객 유입 분석

- Tableau 시각화

<img src = "./img/각 월별 신규 고객수.png">

<img src = "./img/각 월별 신규 고객수 (2).png">


# 월별 재구매율 코호트 분석

### sql

- 월별 Retention 고객 수 테이블 생성
```
CREATE TABLE df_active_users
SELECT
    cohort_month,
    activity_month,
    COUNT(DISTINCT CustomerID) AS active_users -- cohort_month 첫구매 유지 고객 수
FROM
	(SELECT
	    a.CustomerID,
	    a.cohort_month,
	    DATE_FORMAT(b.Date, '%Y-%m') AS activity_month 
	FROM
	    (SELECT 
	        CustomerID,
	        MIN(DATE_FORMAT(Date, '%Y-%m')) AS cohort_month -- 최초 구매 월
	     FROM 
	        df_member
	     GROUP BY 
	        CustomerID) a
	JOIN
	    df_member b 
	ON 
	    a.CustomerID = b.CustomerID -- 각 테이블의 CustomerID가 같은거 기준으로 원본 테이블과 JOIN
	WHERE
	    DATE_FORMAT(b.Date, '%Y-%m') >= a.cohort_month) c -- 원본 테이블의 구매 날짜가 고객의 최초 구매 이후인지 확인
GROUP BY
    cohort_month,
    activity_month
ORDER BY
    cohort_month,
    activity_month;
```
- 월별 Retention Rate 테이블 생성
```
CREATE TABLE df_retention_rate
SELECT 
	a.cohort_month,
    a.activity_month,
    a.active_users,
    round(a.active_users/a.first_active_users, 2) * 100 AS retention_rate
FROM
	(SELECT 
		cohort_month,
	    activity_month,
	    active_users,
	    FIRST_VALUE(active_users) OVER (PARTITION BY cohort_month ORDER BY activity_month) AS first_active_users
	FROM df_active_users) a -- 
ORDER BY
	cohort_month, activity_month;
```

#### 결과 테이블

In [ ]:
df_retention_rate.head()

- Tableau 시각화

<img src = "./img/년_월별 고객 리텐션 차트.png"> <br><br>

<img src = "./img/년_월별 고객 리텐션 표 .png">

- 2010/12
    - 해당 코호트에서 신규고객 유입수와 해당 기간에 유입된 신규고객의 재구매율이 가장 우수했음
    - 또한 2011/11 에 재구매율이 급등하는 모습을 보임

# 가설 1

- 신규고객 유입 수가 가장 우수한 2010/12 코호트와 그 외 기간(2011/01~2011/12) 코호트에서 신규 구매자의 가장 많이 구매한 상품(TOP 5)은 다를 것이다.
    - 이유 : 2010/12 코호트는 다른 코호트 대비 눈에 띄게 좋은 신규고객 유입 수를 보이고 있기 때문

### sql

- 2010/12 신규 구매자의 구매품목 TOP5 

```
WITH retention_filter AS (
    SELECT 
        InvoiceNo,
        CustomerID,
        Category,
        DATE_FORMAT(Date, '%Y-%m') AS Date,
        MIN(DATE_FORMAT(Date, '%Y-%m')) OVER (PARTITION BY CustomerID) AS first_order_month
    FROM df_firstpurchase
)
-- 2010/12그룹
SELECT Category,
	COUNT(DISTINCT CustomerID) AS cnt_customer, -- 해당 카테고리 구매한 고유 고객 수
	ROUND(COUNT(DISTINCT CustomerID) * 100 / (SELECT COUNT(DISTINCT CustomerID) FROM retention_filter WHERE first_order_month = '2010-12'), 2) AS customer_pct,     
        
FROM retention_filter
WHERE Date = '2010-12'
GROUP BY Category
ORDER BY cnt_customer DESC 
LIMIT 5;
```

- 2010/12 외 신규 구매자의 구매품목 TOP5 

```
WITH retention_filter AS (
    SELECT 
        InvoiceNo,
        CustomerID,
        Category,
        DATE_FORMAT(Date, '%Y-%m') AS Date,
        MIN(DATE_FORMAT(Date, '%Y-%m')) OVER (PARTITION BY CustomerID) AS first_order_month
    FROM df_firstpurchase
)
-- 2010/12 이외 그룹
SELECT Category,
	COUNT(DISTINCT CustomerID) AS cnt_customer, -- 해당 카테고리 구매한 고유 고객 수	
	ROUND(COUNT(DISTINCT CustomerID) * 100 / (SELECT COUNT(DISTINCT CustomerID) FROM retention_filter WHERE first_order_month != '2010-12'), 2) AS customer_pct,     
        
FROM retention_filter
WHERE Date != '2010-12'
GROUP BY Category
ORDER BY cnt_customer DESC 
LIMIT 5;
```

### 컬럼 설명

- Category : 상품 카테고리
- cnt_customer : 해당 카테고리 구매 고객 수 
- customer_pct : 해당 카테고리 구매 신규 고객 수 / 해당 기간 유입된 총 신규고객 수

- Tableau 시각화

<img src = "./img/2010 12 비교.png">

- 두 그룹의 Top5 항목에는 몇몇 차이가 있었다

- 2010/12 그룹 Top5 항목에는 'holder', 'christmas', 'bag' 가 있었지만 이외 그룹에는 없었고
- 이외 그룹 Top5 항목에는 'box', 'set', 'cake' 상품이 있었지만 2010/12 그룹에는 없었다.

## 가설 2

- 'bag', 'holder', 'christmas' 상품의 재구매율은  'box', 'set', 'cake' 상품의 재구매율 보다 우수할것이다.
    - 이유 : 재구매율이 우수하였던 2010/12 신규고객들이 다른 기간의 신규 고객들과는 다르게 많이 구매한 상품이 'bag', 'holder', 'christmas' 이기 때문

### sql

```
WITH retail_filter AS (
    SELECT *,
          MIN(DATE_FORMAT(Date, '%Y-%m-01')) OVER (PARTITION BY CustomerID) AS first_order_month, -- 고객별 첫 구매 월
          MIN(Date) OVER (PARTITION BY CustomerID) AS first_order_date  -- 고객별 첫 구매 날짜
    FROM df_member dm 
    WHERE UnitPrice > 0.01  
),
retention_data AS (
    SELECT Category,
           COUNT(DISTINCT CustomerID) AS month0, -- 첫 구매 고객 수
           COUNT(*) AS total_orders, -- 카테고리의 총 구매 건수
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 1 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month1, -- 1개월 후 재구매
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 2 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month2,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 3 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month3,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 4 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month4,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 5 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month5,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 6 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month6,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 7 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month7,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 8 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month8,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 9 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month9,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 10 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month10,
           COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 11 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') AND category = Category THEN CustomerID END) AS month11
    FROM retail_filter
    GROUP BY Category
),
retention_pct AS (
    SELECT Category,
           month0,
           total_orders, -- 카테고리의 총 구매 건수 계산
           ROUND(IF(month1 = 0, 0, month1 / month0 * 100), 2) AS month1_pct,
           ROUND(IF(month2 = 0, 0, month2 / month0 * 100), 2) AS month2_pct,
           ROUND(IF(month3 = 0, 0, month3 / month0 * 100), 2) AS month3_pct,
           ROUND(IF(month4 = 0, 0, month4 / month0 * 100), 2) AS month4_pct,
           ROUND(IF(month5 = 0, 0, month5 / month0 * 100), 2) AS month5_pct,
           ROUND(IF(month6 = 0, 0, month6 / month0 * 100), 2) AS month6_pct,
           ROUND(IF(month7 = 0, 0, month7 / month0 * 100), 2) AS month7_pct,
           ROUND(IF(month8 = 0, 0, month8 / month0 * 100), 2) AS month8_pct,
           ROUND(IF(month9 = 0, 0, month9 / month0 * 100), 2) AS month9_pct,
           ROUND(IF(month10 = 0, 0, month10 / month0 * 100), 2) AS month10_pct,
           ROUND(IF(month11 = 0, 0, month11 / month0 * 100), 2) AS month11_pct
    FROM retention_data
),
average_retention AS (
    SELECT Category,
           total_orders, -- 총 구매 건수 계산
           ROUND((month1_pct + month2_pct + month3_pct + month4_pct + month5_pct + month6_pct + month7_pct + month8_pct + month9_pct + month10_pct + month11_pct) / 11, 2) AS avg_retention_rate
    FROM retention_pct
)
SELECT Category, 
		total_orders, 
		avg_retention_rate
FROM average_retention
ORDER BY avg_retention_rate DESC ;
```

- Tableau 시각화 

<img src = "./img/평균 재구매율 비교.png">

- 2010/12의 Top5 중 'christmas' 를 제외한 'bag' 와 'holder' 상품의 재구매율은 이외 기간의 Top5 상품의 재구매율 보다 우수하였다.
- 따라서 'bag'와 'holder' 상품이 2010/12 코호트의 재구매율을 높이는데 긍정적인 영향을 끼쳤다는 것을 알 수 있다.

## 가설 3 

- 'christmas' 상품은 특정 '월'에 재구매율과 구매량이 증가할 것이다.
    - 이유 : 'christmas' 상품은 '12/25' 을 위해 만들어진 상품이기 때문

### sql

```
WITH retail_filter AS (
    SELECT * ,
          MIN(DATE_FORMAT(Date, '%Y-%m-01')) OVER (PARTITION BY CustomerID) AS first_order_month , -- 고객별 첫구매 월
          MIN(Date) OVER (PARTITION BY CustomerID) AS first_order_date  -- 고객별 첫구매 날짜 
    FROM df_member dm 
    WHERE UnitPrice > 0.01  
) 
SELECT cohort
      ,month0
      ,ROUND(IF(month1 = 0, 0, month1 / month0 * 100) ,2) AS month1_pct -- n개월 후 재구매율
      ,ROUND(IF(month2 = 0, 0, month2 / month0 * 100) ,2) AS month2_pct
      ,ROUND(IF(month3 = 0, 0, month3 / month0 * 100) ,2) AS month3_pct
      ,ROUND(IF(month4 = 0, 0, month4 / month0 * 100) ,2) AS month4_pct
      ,ROUND(IF(month5 = 0, 0, month5 / month0 * 100) ,2) AS month5_pct
      ,ROUND(IF(month6 = 0, 0, month6 / month0 * 100) ,2) AS month6_pct
      ,ROUND(IF(month7 = 0, 0, month7 / month0 * 100) ,2) AS month7_pct
      ,ROUND(IF(month8 = 0, 0, month8 / month0 * 100) ,2) AS month8_pct
      ,ROUND(IF(month9 = 0, 0, month9 / month0 * 100) ,2) AS month9_pct
      ,ROUND(IF(month10 = 0, 0, month10 / month0 * 100) ,2) AS month10_pct
      ,ROUND(IF(month11 = 0, 0, month11 / month0 * 100) ,2) AS month11_pct
FROM (
        SELECT   cohort ,
		         COUNT(DISTINCT CustomerID) AS month0 , -- 재구매 횟수 계산
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 1 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month1 , -- n개월 이후 재구매 횟수
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 2 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month2 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 3 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month3 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 4 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month4 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 5 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month5 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 6 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month6 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 7 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month7 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 8 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month8 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 9 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month9 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 10 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month10 ,
		         COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 11 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month11 ,
				 COUNT(DISTINCT CASE WHEN DATE_ADD(first_order_month, INTERVAL 12 MONTH) = DATE_FORMAT(Date, '%Y-%m-01') THEN CustomerID END) AS month12
		FROM (SELECT * , -- 크리스마스 구매고객 구분
			  	CASE WHEN CustomerID IN ( SELECT DISTINCT CustomerId   -- 첫구매가 크리스마스상품인 고객 추출
		                                       FROM retail_filter
		                            	       WHERE Date = first_order_date 
		                            		   AND category = 'christmas' )
		        THEN 'christmas'
		        ELSE 'non_christmas' 
		        END AS 'cohort'
			    FROM retail_filter)season
		GROUP BY cohort  
     )retention_pct;
```

- Tableau 시각화 <br><br>

- 크리스마스 상품 재구매율 vs 그외 상품 재구매율
<img src = "./img/크리스마스 vs 그 외 상품 (line).png">

<img src = "./img/년_월별 크리스 마스 상품 구매량.png">

- 'christmas' 상품을 구매했던 고객의 'christmas' 상품 재구매율은 11월에 다시 증가했다.
- 'christmas' 상품의 구매량은 9월~12월에 급증하는 모습을 보였다.